Machine Learning CS 6375

Assignment 3

*   Implementing unsupervised K-Means clustering algorithm using Jacardian distance



Group Members:



1.   Anurag Wasankar
2.   Miranda Ferrer


Late Days Used: 1

In [52]:
import numpy as np
import random
import math
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
from tabulate import tabulate

#Fetching data

In [53]:
def read_file(id):
    url = 'https://drive.google.com/uc?export=download&id=' + id
    headers = { 'Accept': 'application/text' }
    r = requests.get(url, headers = headers)
    with open('data.txt', "w") as f:
        f.write(r.text)

In [54]:
#https://drive.google.com/file/d/1twr7O3FgsGY7P81PhG8YSmH36yIt_qAW/view?usp=sharing
read_file('1twr7O3FgsGY7P81PhG8YSmH36yIt_qAW')

#Data Processing

In [55]:
df = pd.read_csv('data.txt', sep='|', header = None, names=['ID', 'Timestamp', 'Tweet'])
df

,ID,Timestamp,Tweet
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,RT @paula_span: The most effective nationwide ...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,The New Old Age Blog: Diabetes Prevention That...
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Well: Comfort Casseroles for Winter Dinners ht...
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...
...,...,...,...
6040,318864599728414720,Mon Apr 01 23:16:59 +0000 2013,Letters: Hpv Vaccines for Boys http://nyti.ms/...
6041,318860789899476992,Mon Apr 01 23:01:51 +0000 2013,Health Law Provision for Small Business Is Del...
6042,318860787605176321,Mon Apr 01 23:01:50 +0000 2013,Global Health: Gay Men in Mexico City Would St...
6043,318856950890766336,Mon Apr 01 22:46:35 +0000 2013,Multiple Sclerosis Balloon Therapy Fails in St...


In [56]:
df.drop(['ID', 'Timestamp'], axis=1, inplace=True)
df

,Tweet
0,Risks in Using Social Media to Spot Signs of M...
1,RT @paula_span: The most effective nationwide ...
2,The New Old Age Blog: Diabetes Prevention That...
3,Well: Comfort Casseroles for Winter Dinners ht...
4,High-Level Knowledge Before Veterans Affairs S...
...,...
6040,Letters: Hpv Vaccines for Boys http://nyti.ms/...
6041,Health Law Provision for Small Business Is Del...
6042,Global Health: Gay Men in Mexico City Would St...
6043,Multiple Sclerosis Balloon Therapy Fails in St...


In [57]:
df.Tweet = df.Tweet.astype(str)

In [58]:
df.Tweet

0       Risks in Using Social Media to Spot Signs of M...
1       RT @paula_span: The most effective nationwide ...
2       The New Old Age Blog: Diabetes Prevention That...
3       Well: Comfort Casseroles for Winter Dinners ht...
4       High-Level Knowledge Before Veterans Affairs S...
                              ...                        
6040    Letters: Hpv Vaccines for Boys http://nyti.ms/...
6041    Health Law Provision for Small Business Is Del...
6042    Global Health: Gay Men in Mexico City Would St...
6043    Multiple Sclerosis Balloon Therapy Fails in St...
6044    The Doctorâs World: Link to Ethical Scandals...
Name: Tweet, Length: 6045, dtype: object

In [59]:
series = df.Tweet
for entry_index in range(len(series)):
  series[entry_index]=series[entry_index].lower()
  series[entry_index] = " ".join(filter(lambda x: x[0] != '@', series[entry_index].split()))
  series[entry_index] = re.sub(r"http\S+", "", series[entry_index])
  series[entry_index] = re.sub(r"www\S+", "", series[entry_index])
  series[entry_index] = series[entry_index].strip()
  length = len(series[entry_index])
  if length > 0:
      if series[entry_index][len(series[entry_index]) - 1] == ':':
          series[entry_index] = series[entry_index][:len(series[entry_index]) - 1]
  series[entry_index] = series[entry_index].replace('#', '')
  series[entry_index] = series[entry_index].translate(str.maketrans('', '', string.punctuation))
  series[entry_index] = " ".join(series[entry_index].split())
df.Tweet = series


In [60]:
df.Tweet

0       risks in using social media to spot signs of m...
1       rt the most effective nationwide diabetes prev...
2       the new old age blog diabetes prevention that ...
3              well comfort casseroles for winter dinners
4       highlevel knowledge before veterans affairs sc...
                              ...                        
6040                        letters hpv vaccines for boys
6041    health law provision for small business is del...
6042    global health gay men in mexico city would sta...
6043    multiple sclerosis balloon therapy fails in study
6044    the doctorâs world link to ethical scandals ...
Name: Tweet, Length: 6045, dtype: object

In [61]:
tweet_list = list(df.Tweet)

In [62]:
def finish(previous, new):
    if len(previous) != len(new):
        return False
    for centroid in range(len(new)):
        if " ".join(new[centroid]) != " ".join(previous[centroid]):
            return False
    return True

In [63]:
k = 3
table=[]
table.append(["Value of K","SSE","Size of each cluster"])
for i in range(5):
  print("Running k-means for k =",str(k))
  centroids = []
  j = 0
  init_centroid_map = dict()
  while j < k:
    centroid = random.randint(0, len(tweet_list) - 1)
    if centroid not in init_centroid_map:
      centroids.append(tweet_list[centroid])
      init_centroid_map[centroid] = True
      j+=1 
          

  iter_count = 0
  previous = []
  
  while (finish(previous, centroids)) == False and (iter_count < 50):
    clusters = dict()
    for tweet_index in range(len(tweet_list)):
      minimum = math.inf
      cluster_idx = -1
      for centroid_entry in range(len(centroids)):
        intersection = set(centroids[centroid_entry]).intersection(tweet_list[tweet_index])
        union = set().union(centroids[centroid_entry], tweet_list[tweet_index])
        dis = 1 - (len(intersection) / len(union))
        if centroids[centroid_entry] == tweet_list[tweet_index]:
          cluster_idx = centroid_entry
          minimum = 0
          break
        if dis < minimum:
          cluster_idx = centroid_entry
          minimum = dis
      if minimum == 1:
        cluster_idx = random.randint(0, len(centroids) - 1)
      clusters.setdefault(cluster_idx, []).append([tweet_list[tweet_index]])
      last_tweet_idx = len(clusters.setdefault(cluster_idx, [])) - 1
      clusters.setdefault(cluster_idx, [])[last_tweet_idx].append(minimum)
    previous = centroids
    
    total_clusters = len(clusters)

    centroids = []
    for cluster_entry in range(total_clusters):
      minimum_sum = math.inf
      centroid_idx = -1
      minimum_dp = []

      for point1 in range(len(clusters[cluster_entry])):
        minimum_dp.append([])
        current_distance = 0
        for point2 in range(len(clusters[cluster_entry])):
          if point1 != point2:
            if point2 < point1:
              dis = minimum_dp[point2][point1]
            else:
              intersection = set(clusters[cluster_entry][point1][0]).intersection(clusters[cluster_entry][point2][0])
              union = set().union(clusters[cluster_entry][point1][0], clusters[cluster_entry][point2][0])
              dis = 1 - (len(intersection) / len(union))

            minimum_dp[point1].append(dis)
            current_distance += dis
          else:
            minimum_dp[point1].append(0)
        if current_distance < minimum_sum:
          minimum_sum = current_distance
          centroid_idx = point1
      centroids.append(clusters[cluster_entry][centroid_idx][0])
    iter_count = iter_count + 1
  sse = 0
  for cluster in range(total_clusters):
        for point in range(len(clusters[cluster])):
            sse = sse + (clusters[cluster][point][1] * clusters[cluster][point][1])
  table_row=[]
  table_row.append(k)
  table_row.append(sse)
  cluster_info = []
  for cluster in range(total_clusters):
      cluster_info.append('Cluster '+ str(cluster+1) + ": " + str(len(clusters[cluster])) + " tweets")
  table_row.append(cluster_info)
  table.append(table_row)
  k = k + 1

Running k-means for k = 3
Running k-means for k = 4
Running k-means for k = 5
Running k-means for k = 6
Running k-means for k = 7


In [64]:
print(tabulate(table, headers='firstrow',tablefmt='orgtbl'))

|   Value of K |     SSE | Size of each cluster                                                                                                                                                             |
|--------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|            3 | 349.081 | ['Cluster 1: 813 tweets', 'Cluster 2: 751 tweets', 'Cluster 3: 4481 tweets']                                                                                                     |
|            4 | 339.526 | ['Cluster 1: 701 tweets', 'Cluster 2: 4547 tweets', 'Cluster 3: 175 tweets', 'Cluster 4: 622 tweets']                                                                            |
|            5 | 307.177 | ['Cluster 1: 817 tweets', 'Cluster 2: 3405 tweets', 'Cluster 3: 783 tweets', 'Cluster 4: 168 tweets', 'Cluster 5: 872 tweets']                       